In [1]:
import pandas as pd

In [11]:
df1 = pd.read_csv("Evaluation_resources_official/sample_ground_truth.csv", index_col=0)
df1.head()

,paragraph_id,answers
question_id,,
1,[],[]
2,[6],['Qatar']
3,[2],['Google']
4,[],[]
5,"[3, 4]",['November 2022']


In [12]:
df2 = pd.read_csv("Evaluation_resources_official/sample_input_paragraph.csv", index_col=0)
df2.head()

,paragraph,theme
id,,
1,Kubernetes is an open-source container orchest...,kubernetes
2,"Google originally designed Kubernetes, but the...",kubernetes
3,"ChatGPT, which stands for Chat Generative Pre-...",ChatGPT
4,ChatGPT was launched as a prototype in Novembe...,ChatGPT
5,The 2022 FIFA World Cup was an international f...,Football world cup


In [14]:
df3 = pd.read_csv("Evaluation_resources_official/sample_input_question.csv", index_col=0)
df3.head()

,question,theme
id,,
1,Who won 2022 Football world cup?,Football world cup
2,Where did the 2022 football world cup take place?,Football world cup
3,Who designed Kubernetes?,Kubernetes
4,When was kubenetes developed?,Kubernetes
5,When was ChatGPT prototype launched?,ChatGPT


In [16]:
df4 = pd.read_csv("Evaluation_resources_official/sample_theme_interval.csv")#, index_col=0)
df4.head()

,theme,start,end
0,Football world cup,1,2
1,Kubernetes,3,4
2,ChatGPT,5,5


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
import json
from sklearn.metrics import accuracy_score
import unicodedata
import timeit
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

In [18]:
stemmer_obj = SnowballStemmer('english')  
lem = WordNetLemmatizer()

def lemmatizer_stemmer(text):
    list_sentences = text.split(".")
    new_lines = []
    for line in list_sentences:
        list_words = line.split(" ")
        # print(line)
        new_line = []
        for word in list_words:
            new_line.append(stemmer_obj.stem(lem.lemmatize(word)))
        new_lines.append(" ".join(new_line))
        # print(new_line)
    return ".".join(new_lines)

def remove_stopwords(text):
    list_sentences = text.split(".")
    new_lines = []
    for line in list_sentences:
        list_words = line.split(" ")
        new_lines.append(" ".join([word.lower() for word in list_words if word not in stopwords.words('english')]))
    return  ".".join(new_lines)

def remove_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_punctuations(text):
    return text.replace('?',"")

def preprocess_text(text: str) -> str :
    return lemmatizer_stemmer(remove_stopwords(remove_punctuations(remove_accents(text))))

In [101]:
# NOTE: make changes in this cell only.
df_paras = pd.read_csv("Evaluation_resources_official/sample_input_paragraph.csv")
def pred_theme_ans(questions, pred_out):
  #-----------------------------
  # Warmup module for a theme model goes here.
  #------------------------------
  theme = questions[0]["theme"]
    
  # Vectorizer & Cos Similarity :-
  vectorizer = TfidfVectorizer()
  all_paras = list(df_paras[df_paras.theme.str.lower()==theme.lower()].paragraph.unique())
  paragraphs_vec = vectorizer.fit_transform([preprocess_text(i) for i in all_paras])
  # print(type(paragraphs_vec))
  print(paragraphs_vec.shape)
  # print("***********************")
  print("---------------------------")

  for question in questions:
    #-------------------------------------   
    # add your prediction methodology here.
    #-------------------------------------
    
    # TfIDf vectorizer Method :-
    ans = {}
    ans["question_id"] = question["id"]
    quesn_vec = vectorizer.transform([question["question"]])
    print(quesn_vec.reshape(quesn_vec.shape[1], 1).shape)
    cos_prods = np.dot(paragraphs_vec, quesn_vec.reshape((quesn_vec.shape[1], 1)))
    most_likely_para_idx = df_paras[df_paras.paragraph == all_paras[cos_prods.argmax()]].id.item()
    print(cos_prods.shape)
    print(all_paras[cos_prods.argmax()])
    print(most_likely_para_idx)
    
    # Dummy method.
#     ans = {}
#     ans["question_id"] = question["id"]
#     if theme == "Kubernetes":
#         # If no prediction for a paragraph, predict -1 for paragraph prediction
#         # and empty string for answers.
#         ans["paragraph_id"] = 2
#         ans["answers"] = "Google"
#     elif theme == "ChatGPT":
#         ans["paragraph_id"] = 4
#         ans["answers"] = "2022"
#     elif theme == "Football world cup":
#         ans["paragraph_id"] = 6
#         ans["answers"] = "Qatar"
#     pred_out.append(ans)

In [102]:
# All theme prediction.
questions = json.loads(pd.read_csv("Evaluation_resources_official/sample_input_question.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("Evaluation_resources_official/sample_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('Evaluation_resources_official/sample_output_prediction.csv', index=False)

(2, 24)
---------------------------
(24, 1)
(2, 1)
The 2022 FIFA World Cup was an international football tournament contested by the men's national teams of FIFA's member associations and 22nd edition of the FIFA World Cup.
5
(24, 1)
(2, 1)
The 2022 FIFA World Cup was an international football tournament contested by the men's national teams of FIFA's member associations and 22nd edition of the FIFA World Cup.
5
(2, 21)
---------------------------
(21, 1)
(2, 1)
Google originally designed Kubernetes, but the Cloud Native Computing Foundation now maintains the project.
2
(21, 1)
(2, 1)
Kubernetes is an open-source container orchestration system for automating software deployment, scaling, and management.
1
(2, 16)
---------------------------
(16, 1)
(2, 1)
ChatGPT was launched as a prototype in November 2022, and quickly garnered attention for its detailed responses.
4


In [103]:
df3.head()

,question,theme
id,,
1,Who won 2022 Football world cup?,Football world cup
2,Where did the 2022 football world cup take place?,Football world cup
3,Who designed Kubernetes?,Kubernetes
4,When was kubenetes developed?,Kubernetes
5,When was ChatGPT prototype launched?,ChatGPT


In [106]:
df2

,paragraph,theme
id,,
1,Kubernetes is an open-source container orchest...,kubernetes
2,"Google originally designed Kubernetes, but the...",kubernetes
3,"ChatGPT, which stands for Chat Generative Pre-...",ChatGPT
4,ChatGPT was launched as a prototype in Novembe...,ChatGPT
5,The 2022 FIFA World Cup was an international f...,Football world cup
6,It took place in Qatar from 20 November to 18 ...,Football world cup


In [38]:
list(df_paras[df_paras.theme=='Kubernetes'].paragraph.unique())

[]

In [36]:
theme_intervals

[{'theme': 'Football world cup', 'start': 1, 'end': 2},
 {'theme': 'Kubernetes', 'start': 3, 'end': 4},
 {'theme': 'ChatGPT', 'start': 5, 'end': 5}]

In [96]:
np.dot(np.array([[2, 4], [0, 6], [3, 3]]), np.array([11, 1]).reshape((2, 1)))

array([[26],
       [ 6],
       [36]])

In [60]:
np.array([[2, 4], [0, 6], [3, 3]]).shape

(3, 2)

In [58]:
np.array([11, 1]).shape

(2,)

In [65]:
np.array([11, 1]).reshape((1, 2)).squeeze().shape

(2,)